In [32]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import  PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda, RunnablePassthrough

from langchain_core.tools import tool

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from langchain_core.tools import InjectedToolArg
from typing import Annotated

import os
import requests

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['HF_API_KEY'] = os.getenv('HF_API_KEY')


In [3]:
# create tool

@tool
def multply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a + b

In [4]:
multply.invoke({"a":3, "b":4}) # 12

7

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0, max_tokens=1000)

In [6]:
llm_with_tools = llm.bind_tools([multply])

In [7]:
llm_with_tools.invoke("How are you").content

"I'm here and ready to help! How can I assist you today?"

In [8]:
query = HumanMessage("multiply 3 with 10")


In [9]:
messages = [query]

In [10]:
messages

[HumanMessage(content='multiply 3 with 10', additional_kwargs={}, response_metadata={})]

In [11]:
result = llm_with_tools.invoke(messages)

In [12]:
messages.append(result)
messages 

[HumanMessage(content='multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CSR625rFV5uEt7P3O3NxR68A', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 53, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e6d9db58-f093-459d-a589-a2da6ed67881-0', tool_calls=[{'name': 'multply', 'args': {'a': 3, 'b': 10}, 'id': 'call_CSR625rFV5uEt7P3O3NxR68A', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 19, 'total_tokens': 72, 'input_token_details': {'audio': 0, 'cache

In [13]:
result.tool_calls[0]

{'name': 'multply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_CSR625rFV5uEt7P3O3NxR68A',
 'type': 'tool_call'}

In [14]:
result.tool_calls[0]['args']

{'a': 3, 'b': 10}

In [15]:
tool_result = multply.invoke(result.tool_calls[0]['args'])

In [16]:
tool_result

13

In [17]:
messages.append(tool_result)

In [18]:
messages

[HumanMessage(content='multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CSR625rFV5uEt7P3O3NxR68A', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 53, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e6d9db58-f093-459d-a589-a2da6ed67881-0', tool_calls=[{'name': 'multply', 'args': {'a': 3, 'b': 10}, 'id': 'call_CSR625rFV5uEt7P3O3NxR68A', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 19, 'total_tokens': 72, 'input_token_details': {'audio': 0, 'cache

In [19]:
# Currency Conversion Tool


In [68]:
@tool
def get_conversion_rate(from_currency: str, to_currency: str) -> float:
    """Get the conversion rate from one currency to another."""
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{from_currency}/{to_currency}'

    response = requests.get(url)
    data = response.json()
    return data

In [69]:
get_conversion_rate.invoke({"from_currency": "USD", "to_currency": "INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745712001,
 'time_last_update_utc': 'Sun, 27 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745798401,
 'time_next_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4301}

In [70]:
@tool
def convert_currency(base_currency_value: int, conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """Convert a currency value to another currency using the conversion rate."""
    return base_currency_value * conversion_rate


In [71]:
convert_currency.invoke({"base_currency_value": 100, "conversion_rate": 82.5}) # 8250.0

8250.0

In [83]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0, max_tokens=1000)
llm_with_tools = llm.bind_tools([get_conversion_rate, convert_currency])

In [89]:
messages = [
    HumanMessage("What is the conversion rate from USD to INR?, based on that can you convert 100 USD to INR")
    ]

In [90]:
ai_message= llm_with_tools.invoke(messages)

In [91]:
messages.append(ai_message)

In [92]:
ai_message.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'from_currency': 'USD', 'to_currency': 'INR'},
  'id': 'call_DUWQaLXwhu43YLxTttAE0xOz',
  'type': 'tool_call'}]

In [64]:
import json
for tool_call in ai_message.tool_calls:
    # print(tool_call)
    if tool_call['name'] == 'get_conversion_rate':
        tool_message_1 = get_conversion_rate.invoke(tool_call)
        print(json.loads(tool_message_1.content)['conversion_rate'])

        messages.append(tool_message_1)

    if tool_call['name'] == 'convert_currency':
        tool_call['args']['conversion_rate'] = json.loads(tool_message_1.content)['conversion_rate']
        tool_message_2 = convert_currency.invoke(tool_call)
        messages.append(tool_message_2)
    

85.4301


In [65]:
messages

[HumanMessage(content='What is the conversion rate from USD to INR?, based on that can you convert 100 USD to INR?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BR5R2FCFAsMPxEXJodSfkxM7', 'function': {'arguments': '{"from_currency":"USD","to_currency":"INR"}', 'name': 'get_conversion_rate'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 108, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f5fa8428-214e-426f-8d99-8b1fe33bcc19-0', tool_calls=[{'name': 'get_conversion_rate', 'args': {'from_currency': 'USD', 'to_currency': 'INR'}, 'id': 'call_BR5R2FCFAsMP